Note that most of this notebook directly come from online source, the code layout is pretty bad, might take a while to understand what the original author is writing 

https://scikit-learn.org/stable/modules/classes.html#

In [115]:
import csv                               # csv reader
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from random import shuffle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\edton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import random
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [67]:
# load data from a file and append it to the rawData
def loadGoldData(shuffle = True):
    loader = DataLoader()
    dec_data = loader.load_gold_data(_type = 'dec', neg_polarity=True, pos_polarity=True)
    truth_data = loader.load_gold_data(_type = 'truth', neg_polarity=True, pos_polarity=True)

    data = []

    for i in range(len(dec_data)):
        data.append((dec_data[i], 'fake'))
        data.append((truth_data[i], 'real'))

    if shuffle:
        random.shuffle(data)
    return data

# load data from a file and append it to the rawData
def loadData(path, Text=None):
    with open(path, encoding="utf8") as f:
        reader = csv.reader(f)

        next(reader)
        for line in reader:
            (Id, Rating, verified_Purchase, product_Category, Text, Label) = parseReview(line)
            rawData.append((Id, Rating, verified_Purchase, product_Category, Text, Label))
            #preprocessedData.append((Id, preProcess(Text), Label))

def sentiment_split(data):
    strong = []
    weak = []
    analyser = SentimentIntensityAnalyzer()

    for i in range(len(data)):
        vadersenti = analyser.polarity_scores(data[i][4])
        if abs(vadersenti['compound']) > 0.5:
            strong.append(data[i])
        else:
            weak.append(data[i])
    
    return strong, weak

def splitData(percentage, data):
    dataSamples = len(data)
    halfOfData = int(len(data)/2)
    trainingSamples = int((percentage*dataSamples)/2)
    for (_, Rating, verified_Purchase, product_Category, Text, Label) in data[:trainingSamples] + data[halfOfData:halfOfData+trainingSamples]:
        trainData.append((toFeatureVector(preProcess(Text)),Label))
        # trainData.append((toFeatureVector(Rating, verified_Purchase, product_Category, preProcess(Text)),Label))
    for (_, Rating, verified_Purchase, product_Category, Text, Label) in data[trainingSamples:halfOfData] + data[halfOfData+trainingSamples:]:
        testData.append((toFeatureVector(preProcess(Text)),Label))
# QUESTION 1


# Convert line from input file into an id/text/label tuple
def parseReview(reviewLine):
    # Should return a triple of an integer, a string containing the review, and a string indicating the label
    s=""
    if reviewLine[1]=="__label1__":
        s = "fake"
    else: 
        s = "real"
    return (reviewLine[0], reviewLine[2], reviewLine[3],reviewLine[4], reviewLine[8], s)
# TEXT PREPROCESSING AND FEATURE VECTORIZATION
# Input: a string of one review
table = str.maketrans({key: None for key in string.punctuation})


def preProcess(text):
    # Should return a list of tokens
    lemmatizer = WordNetLemmatizer()
    filtered_tokens=[]
    lemmatized_tokens = []
    stop_words = set(stopwords.words('english'))
    text = text.translate(table)
    for w in text.split(" "):

        ######################## try without removing ################################################################
        lemmatized_tokens.append(lemmatizer.lemmatize(w.lower()))
        filtered_tokens = [' '.join(l) for l in nltk.bigrams(lemmatized_tokens)] + lemmatized_tokens

        ######################## try keep stopwords only ################################################################
        # if w in stop_words:
        #     lemmatized_tokens.append(lemmatizer.lemmatize(w.lower()))
        # filtered_tokens = [' '.join(l) for l in nltk.ngrams(lemmatized_tokens, n=1)] + lemmatized_tokens

        ######################## try removing ################################################################
        # if w not in stop_words:
        #     lemmatized_tokens.append(lemmatizer.lemmatize(w.lower()))
        # filtered_tokens = [' '.join(l) for l in nltk.bigrams(lemmatized_tokens)] + lemmatized_tokens
    return filtered_tokens
featureDict = {} # A global dictionary of features


# def toFeatureVector(Rating, verified_Purchase, product_Category, tokens):
def toFeatureVector(tokens):

    localDict = {}
    
# #Rating

#     #print(Rating)
#     featureDict["R"] = 1   
#     localDict["R"] = Rating

# #Verified_Purchase
  
#     featureDict["VP"] = 1
            
#     if verified_Purchase == "N":
#         localDict["VP"] = 0
#     else:
#         localDict["VP"] = 1

# #Product_Category

    
#     if product_Category not in featureDict:
#         featureDict[product_Category] = 1
#     else:
#         featureDict[product_Category] = +1
            
#     if product_Category not in localDict:
#         localDict[product_Category] = 1
#     else:
#         localDict[product_Category] = +1
            
            
#Text        

    for token in tokens:
        if token not in featureDict:
            featureDict[token] = 1
        else:
            featureDict[token] = +1
            
        if token not in localDict:
            localDict[token] = 1
        else:
            localDict[token] = +1
    
    return localDict

In [28]:
import os
from utils.configs import config
from utils.loader import DataLoader



In [29]:
path = os.path.join(config['base_path'], config['amazon_path'], 'train')
reviewPath = DataLoader.list_all_txt_files(path)[0]
print(reviewPath)

data\amazon_reviews\train\amazon_reviews.txt


In [107]:
# MAIN

# loading reviews
rawData = []          # the filtered data from the dataset file (should be 21000 samples)
preprocessedData = [] # the preprocessed reviews (just to see how your preprocessing is doing)
trainData = []        # the training data as a percentage of the total dataset (currently 80%, or 16800 samples)
testData = []         # the test data as a percentage of the total dataset (currently 20%, or 4200 samples)

# the output classes
fakeLabel = 'fake'
realLabel = 'real'

# references to the data files
path = os.path.join(config['base_path'], config['amazon_path'], 'train')
reviewPath = DataLoader.list_all_txt_files(path)[0]

## Do the actual stuff
# We parse the dataset and put it in a raw data list
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing the dataset...",sep='\n')

loadData(reviewPath) 
strong, weak = sentiment_split(rawData)

# We split the raw dataset into a set of training data and a set of test data (80/20)
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing training and test data...",sep='\n')

print(len(strong), len(weak))

Now 0 rawData, 0 trainData, 0 testData
Preparing the dataset...
Now 15750 rawData, 0 trainData, 0 testData
Preparing training and test data...
12598 3152


In [109]:
splitData(0.8, strong)
# We print the number of training samples and the number of features
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')

Now 15750 rawData, 10078 trainData, 2520 testData
Training Samples: 
10078
Features: 
365196


In [117]:
trainData[0]

({'i bought': 1,
  'bought this': 1,
  'this for': 1,
  'for my': 1,
  'my husband': 1,
  'husband to': 1,
  'to replace': 1,
  'replace an': 1,
  'an old': 1,
  'old calculator': 1,
  'calculator watch': 1,
  'watch he': 1,
  'he had': 1,
  'had in': 1,
  'in high': 1,
  'high schoolyes': 1,
  'schoolyes he': 1,
  'he still': 1,
  'still like': 1,
  'like them': 1,
  'them he': 1,
  'he thought': 1,
  'thought it': 1,
  'it wa': 1,
  'wa great': 1,
  'great and': 1,
  'and i': 1,
  'i thought': 1,
  'wa a': 1,
  'a good': 1,
  'good price': 1,
  'price for': 1,
  'for the': 1,
  'the watch': 1,
  'watch so': 1,
  'so far': 1,
  'far it': 1,
  'it work': 1,
  'work just': 1,
  'just fine': 1,
  'i': 1,
  'bought': 1,
  'this': 1,
  'for': 1,
  'my': 1,
  'husband': 1,
  'to': 1,
  'replace': 1,
  'an': 1,
  'old': 1,
  'calculator': 1,
  'watch': 1,
  'he': 1,
  'had': 1,
  'in': 1,
  'high': 1,
  'schoolyes': 1,
  'still': 1,
  'like': 1,
  'them': 1,
  'thought': 1,
  'it': 1,
  'wa'

In [110]:
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# analyser = SentimentIntensityAnalyzer()

# def print_sentiment_scores(tweets):
#     vadersenti = analyser.polarity_scores(tweets)
#     return vadersenti['compound']
#     # return ([vadersenti['pos'], vadersenti['neg'], vadersenti['neu'], vadersenti['compound']])

In [114]:
analyser = SentimentIntensityAnalyzer()

for i in range(100):
    vadersenti = analyser.polarity_scores(rawData[i][4])
    print(vadersenti)

{'neg': 0.0, 'neu': 0.771, 'pos': 0.229, 'compound': 0.891}
{'neg': 0.046, 'neu': 0.799, 'pos': 0.156, 'compound': 0.7173}
{'neg': 0.067, 'neu': 0.89, 'pos': 0.043, 'compound': -0.4153}
{'neg': 0.031, 'neu': 0.797, 'pos': 0.172, 'compound': 0.6677}
{'neg': 0.039, 'neu': 0.778, 'pos': 0.183, 'compound': 0.8993}
{'neg': 0.0, 'neu': 0.598, 'pos': 0.402, 'compound': 0.8952}
{'neg': 0.0, 'neu': 0.777, 'pos': 0.223, 'compound': 0.8674}
{'neg': 0.032, 'neu': 0.857, 'pos': 0.112, 'compound': 0.9531}
{'neg': 0.06, 'neu': 0.719, 'pos': 0.221, 'compound': 0.9658}
{'neg': 0.045, 'neu': 0.729, 'pos': 0.226, 'compound': 0.8733}
{'neg': 0.0, 'neu': 0.717, 'pos': 0.283, 'compound': 0.9158}
{'neg': 0.0, 'neu': 0.767, 'pos': 0.233, 'compound': 0.8173}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.067, 'neu': 0.62, 'pos': 0.313, 'compound': 0.8447}
{'neg': 0.074, 'neu': 0.761, 'pos': 0.166, 'compound': 0.9651}
{'neg': 0.0, 'neu': 0.627, 'pos': 0.373, 'compound': 0.8934}
{'neg': 0.0, 'ne

In [72]:
# TRAINING AND VALIDATING OUR CLASSIFIER
def trainClassifier(trainData):
    print("Training Classifier...")
    pipeline =  Pipeline([('svc', LinearSVC())])
    return SklearnClassifier(pipeline).train(trainData)


def predictLabels(reviewSamples, classifier):
    return classifier.classify_many(map(lambda t: t[0], reviewSamples))
    

def crossValidate(dataset, folds):
    shuffle(dataset)
    cv_results = []
    foldSize = int(len(dataset)/folds)
    for i in range(0,len(dataset),foldSize):
        classifier = trainClassifier(dataset[:i]+dataset[foldSize+i:])
        y_pred = predictLabels(dataset[i:i+foldSize],classifier)
        a = accuracy_score(list(map(lambda d : d[1], dataset[i:i+foldSize])), y_pred)
        (p,r,f,_) = precision_recall_fscore_support(list(map(lambda d : d[1], dataset[i:i+foldSize])), y_pred, average ='macro')
        #print(a,p,r,f)
        cv_results.append((a,p,r,f))
    cv_results = (np.mean(np.array(cv_results),axis=0))
    return cv_results

In [32]:
rawGold = loadGoldData(shuffle = True)

In [33]:
testGold = []
for (Text, Label) in rawGold:
    testGold.append((toFeatureVector(preProcess(Text)),Label))

In [97]:
####################################################
# run this to change the testData

test_Data = testGold
# test_Data = testData


####################################################

In [98]:
# additional classifiers
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

def MNB_Classifier(trainData):
    print("Training Classifier...")
    MNB_classifier = SklearnClassifier(MultinomialNB())
    return MNB_classifier.train(trainData)


def BernoulliNB_Classifier(trainData):
    print("Training Classifier...")
    BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
    return BernoulliNB_classifier.train(trainData)


def LogisticRegression_Classifier(trainData):
    print("Training Classifier...")
    LogisticRegression_classifier = SklearnClassifier(LogisticRegression(max_iter=1000))
    return LogisticRegression_classifier.train(trainData)

def SGDClassifier_Classifier(trainData):
    print("Training Classifier...")
    SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
    return SGDClassifier_classifier.train(trainData)


In [99]:
# print("Mean of cross-validations (Accuracy, Precision, Recall, Fscore): ", crossValidate(trainData, 10))


In [100]:
#  TEST DATA
classifier = trainClassifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.625625
Precision:  0.6287687687687687
Recall:  0.625625
f1-score:  0.6233259641365757


In [101]:
#  TEST DATA
classifier = MNB_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.58
Precision:  0.5805482313999659
Recall:  0.58
f1-score:  0.579284125645043


In [102]:
#  TEST DATA
classifier = BernoulliNB_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.578125
Precision:  0.5786441744327788
Recall:  0.578125
f1-score:  0.5774275904569064


In [105]:
#  TEST DATA
classifier = LogisticRegression_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.64625
Precision:  0.6495026162957851
Recall:  0.64625
f1-score:  0.6443154092807912


In [104]:
#  TEST DATA
classifier = SGDClassifier_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.61625
Precision:  0.6162529063226581
Recall:  0.61625
f1-score:  0.6162476015475096


In [42]:
test_samples = testData
pred = predictLabels(test_samples, classifier)
summ = 0
for i in range(len(test_samples)):
    correct = np.where(test_samples[i][1] == pred[i], 1, 0)
    # print(f'True: {test_samples[i][1]}, Predict: {pred[i]},   {correct}')
    summ += correct

print(f'Acc: {summ/len(test_samples)}')


Acc: 0.6787301587301587


In [43]:
type(trainData)

list

In [44]:
print(trainData[1][0], trainData[1][1])

{'i didnt': 1, 'didnt really': 1, 'really like': 1, 'like putting': 1, 'putting the': 1, 'the studio': 1, 'studio rta': 1, 'rta atower': 1, 'atower corner': 1, 'corner wood': 1, 'wood computer': 1, 'computer desk': 1, 'desk with': 1, 'with hutch': 1, 'hutch in': 1, 'in pewter': 1, 'pewter and': 1, 'and cherry': 1, 'cherry together': 1, 'together but': 1, 'but it': 1, 'it work': 1, 'work great': 1, 'great and': 1, 'and fit': 1, 'fit into': 1, 'into my': 1, 'my small': 1, 'small office': 1, 'office corner': 1, 'i': 1, 'didnt': 1, 'really': 1, 'like': 1, 'putting': 1, 'the': 1, 'studio': 1, 'rta': 1, 'atower': 1, 'corner': 1, 'wood': 1, 'computer': 1, 'desk': 1, 'with': 1, 'hutch': 1, 'in': 1, 'pewter': 1, 'and': 1, 'cherry': 1, 'together': 1, 'but': 1, 'it': 1, 'work': 1, 'great': 1, 'fit': 1, 'into': 1, 'my': 1, 'small': 1, 'office': 1} fake


In [45]:
type(trainData[1][0])

dict

In [46]:
classifier = trainClassifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))

Training Classifier...


In [47]:
from sklearn.neural_network import MLPClassifier

def MLPClassifier_Classifier(trainData):
    print("Training Classifier...")
    MNB_classifier = SklearnClassifier(MLPClassifier(hidden_layer_sizes=(100,), max_iter=50, verbose=True))
    return MNB_classifier.train(trainData)

#  TEST DATA
classifier = MLPClassifier_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
Iteration 1, loss = 0.61347991
Iteration 2, loss = 0.19353398
Iteration 3, loss = 0.04392985


C:\Users\edton\miniconda3\envs\SNLP\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


accuracy:  0.6325
Precision:  0.6359811165845648
Recall:  0.6325
f1-score:  0.630132850241546


In [ ]:
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

accuracy:  0.6628571428571428
Precision:  0.6633553245863717
Recall:  0.6628571428571428
f1-score:  0.6627652423497419
